# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import hashlib
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from IPython.display import display

# LOAD DATA

In [3]:
file_path = "wwlLancMsc_data.xlsx"
sheet_name = "in"  # change if different

df = pd.read_excel(file_path, sheet_name=sheet_name)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

Shape: (41846, 101)
Columns: ['site_national_code', 'site_description', 'site_local_code', 'Admission_Date', 'admission_date_dt', 'discharge_date_dt', 'specialty_local_code', 'specialty_spec_code', 'specialty_spec_desc', 'ward_code_admission', 'ward_name_admission', 'ward_code_discharge', 'ward_name_discharge', 'date_of_birth_dt', 'date_of_death_dt', 'ethnic_origin_description', 'duration_elective_wait', 'patient_age_on_admission', 'patient_age_on_discharge', 'spell_episode_los', 'discharge_delay_reason_national_code', 'discharge_delay_reason_description', 'spell_dominant_proc', 'spell_primary_diagnosis', 'spell_secondary_diagnosis', 'specialty_division', 'specialty_directorate', 'hrg_group', 'hrg_sub_group', 'delayed_discharges_flag', 'delayed_discharges_no_of_days', 'readmission_flag_28_days', 'readmission_flag_30_days', 'social_worker_date_time_referred', 'discharge_letter_sent', 'discharge_letter_sent_in_24hrs', 'inpatient_death_flag', 'spell_days_elective', 'spell_days_non_electiv

# CREATE PATIENT HASH

In [4]:
if "PatientID" in df.columns:
    df["Patient_Hash"] = df["PatientID"].astype(str).apply(
        lambda x: hashlib.sha256(x.encode()).hexdigest()
    )

# CREATE MAPPING TABLES + DROP REDUNDANT TEXT COLUMNS

In [5]:
# 4.1 – Site mapping: site_national_code ↔ site_description + site_local_code
if {"site_national_code", "site_description", "site_local_code"}.issubset(df.columns):
    site_map = df[["site_national_code", "site_description", "site_local_code"]].drop_duplicates()
    site_map.to_csv("mapping_sites.csv", index=False)
    print("Saved site mapping → mapping_sites.csv")
else:
    print("Site columns not complete, skipping site mapping.")

# 4.2 – Specialty mapping: specialty_local_code ↔ specialty_spec_code ↔ specialty_spec_desc
if {"specialty_local_code", "specialty_spec_code", "specialty_spec_desc"}.issubset(df.columns):
    spec_map = df[["specialty_local_code", "specialty_spec_code", "specialty_spec_desc"]].drop_duplicates()
    spec_map.to_csv("mapping_specialty.csv", index=False)
    print("Saved specialty mapping → mapping_specialty.csv")
else:
    print("Specialty columns not complete, skipping specialty mapping.")

# 4.3 – Ward mapping: ward_code ↔ ward_name (admission & discharge)
ward_maps = []

if {"ward_code_admission", "ward_name_admission"}.issubset(df.columns):
    ward_adm_map = df[["ward_code_admission", "ward_name_admission"]].drop_duplicates()
    ward_adm_map.to_csv("mapping_ward_admission.csv", index=False)
    ward_maps.append("mapping_ward_admission.csv")

if {"ward_code_discharge", "ward_name_discharge"}.issubset(df.columns):
    ward_dis_map = df[["ward_code_discharge", "ward_name_discharge"]].drop_duplicates()
    ward_dis_map.to_csv("mapping_ward_discharge.csv", index=False)
    ward_maps.append("mapping_ward_discharge.csv")

if ward_maps:
    print("Saved ward mappings →", ward_maps)
else:
    print("Ward mapping not created (columns missing).")

# 4.4 – Drop redundant mapped columns for better representation
cols_to_drop = [
    # site description & local code (we keep site_national_code)
    "site_description",
    "site_local_code",
    # specialty: keep specialty_spec_code (numeric), drop others
    "specialty_local_code",
    "specialty_spec_desc",
    # ward names: keep codes, drop names
    "ward_name_admission",
    "ward_name_discharge",
]

df = df.drop(columns=cols_to_drop, errors="ignore")

print("After dropping mapped text columns, shape:", df.shape)
print("Remaining columns:", len(df.columns))


Saved site mapping → mapping_sites.csv
Saved specialty mapping → mapping_specialty.csv
Saved ward mappings → ['mapping_ward_admission.csv', 'mapping_ward_discharge.csv']
After dropping mapped text columns, shape: (41846, 95)
Remaining columns: 95


# IDENTIFY NUMERIC & CATEGORICAL COLUMNS

In [6]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numeric columns:", len(num_cols))
print("Categorical columns:", len(cat_cols))

Numeric columns: 56
Categorical columns: 37


# DROP COLUMNS THAT ARE COMPLETELY EMPTY

In [7]:
all_missing = [c for c in df.columns if df[c].isna().all()]

if all_missing:
    print("Dropping completely missing columns:", all_missing)
    df = df.drop(columns=all_missing)
    num_cols = [c for c in num_cols if c not in all_missing]
    cat_cols = [c for c in cat_cols if c not in all_missing]
else:
    print("No completely empty columns.")

Dropping completely missing columns: ['discharge_delay_reason_national_code', 'social_worker_date_time_referred']


# IMPUTE MISSING VALUES

In [8]:
# Numeric → median
for c in num_cols:
    if df[c].notna().any():
        df[c] = df[c].fillna(df[c].median())

# Categorical → mode (after casting to str)
for c in cat_cols:
    df[c] = df[c].astype(str)
    mode_val = df[c].mode()
    if not mode_val.empty:
        df[c] = df[c].fillna(mode_val[0])

print("Top missing after imputation:")
print(df.isnull().sum().sort_values(ascending=False).head(20))

Top missing after imputation:
Arrival_Date                          30382
Admission_Date                            0
admission_date_dt                         0
discharge_date_dt                         0
site_national_code                        0
ward_code_admission                       0
ward_code_discharge                       0
date_of_birth_dt                          0
date_of_death_dt                          0
ethnic_origin_description                 0
duration_elective_wait                    0
patient_age_on_admission                  0
specialty_spec_code                       0
spell_episode_los                         0
discharge_delay_reason_description        0
spell_dominant_proc                       0
spell_primary_diagnosis                   0
spell_secondary_diagnosis                 0
specialty_division                        0
specialty_directorate                     0
dtype: int64


# OPTIONAL BINS (AGE / LOS) IF PRESENT

In [9]:
if "Age" in df.columns:
    df["Age_Group"] = pd.cut(
        df["Age"],
        bins=[0, 12, 18, 35, 60, 120],
        labels=["Child", "Teen", "YoungAdult", "Adult", "Senior"],
        include_lowest=True
    )

if "LOS" in df.columns:
    df["LOS_Group"] = pd.cut(
        df["LOS"],
        bins=[0, 3, 7, 30, 1000],
        labels=["Short", "Medium", "Extended", "VeryLong"],
        include_lowest=True
    )

# OUTLIER CAPPING(IQR)

In [10]:
numeric_for_outlier = [
    c for c in df.select_dtypes(include=[np.number]).columns
    if df[c].nunique() > 1
]

for c in numeric_for_outlier:
    df[c] = df[c].astype(float)
    q1 = df[c].quantile(0.25)
    q3 = df[c].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df[c] = np.where(df[c] < lower, lower, df[c])
    df[c] = np.where(df[c] > upper, upper, df[c])

print("Outlier capping complete.")

Outlier capping complete.


# ENCODE CATEGORICAL COLUMNS (TEXT → NUMERIC)

In [11]:
identifier_cols = [
    "PatientID",
    "Patient_Hash",
    "ID",              # if present
]

cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
cat_to_encode = [c for c in cat_cols if c not in identifier_cols]

encoded_df = pd.DataFrame(index=df.index)

for c in cat_to_encode:
    nunq = df[c].nunique()

    if nunq <= 10:
        dummies = pd.get_dummies(df[c].astype(str), prefix=c)
        encoded_df = pd.concat([encoded_df, dummies], axis=1)
    else:
        enc = OrdinalEncoder(dtype=int)
        arr = df[c].astype(str).values.reshape(-1, 1)
        encoded_vals = enc.fit_transform(arr).astype(int).ravel()
        encoded_df[c + "_ord"] = encoded_vals

        mapping_df = pd.DataFrame({c: enc.categories_[0]})
        mapping_df["ordinal"] = range(len(mapping_df))
        mapping_df.to_csv(f"mapping_{c}.csv", index=False)

print("Encoding complete. Encoded shape:", encoded_df.shape)

Encoding complete. Encoded shape: (41846, 75)


# BUILD FINAL NUMERIC DATASET & SCALE

In [12]:
final_numeric = df.select_dtypes(include=[np.number]).copy()
cleaned = pd.concat([final_numeric, encoded_df], axis=1)

cleaned = cleaned.astype(float)

scaler = StandardScaler()
cleaned[cleaned.columns] = scaler.fit_transform(cleaned)

print("Final model-ready shape:", cleaned.shape)

Final model-ready shape: (41846, 129)


# SAVE FILES

In [13]:
df.to_csv("cleaned_nhs_for_report.csv", index=False)
cleaned.to_csv("cleaned_nhs_data_for_model.csv", index=False)

print("Saved cleaned_nhs_for_report.csv")
print("Saved cleaned_nhs_data_for_model.csv")


Saved cleaned_nhs_for_report.csv
Saved cleaned_nhs_data_for_model.csv


# SHOW OUTPUT

In [14]:
print("For report (clean, mapped, no redundant text):")
display(df.head())

print("For model (numeric, encoded, scaled):")
display(cleaned.head())

For report (clean, mapped, no redundant text):


,site_national_code,Admission_Date,admission_date_dt,discharge_date_dt,specialty_spec_code,ward_code_admission,ward_code_discharge,date_of_birth_dt,date_of_death_dt,ethnic_origin_description,...,source_of_ref_description,presenting_complaint,acuity_code,inj_or_ail,NEWS2,attend_dis_description,ae_unplanned_attendance,location,ID,Deprivation Decile
0,RRF01,2023-07-15,15/07/2023 13:57:00,15/07/2023 16:00:00,104.0,GAST,GAST,1947-12-12 00:00:00,nan,British (White),...,nan,nan,3.0,nan,1.0,nan,1.0,nan,49591.0,3.0
1,RRF53,2023-07-05,2023-05-07 08:37:00,2023-05-07 10:08:00,110.0,OAU,OAU,27/11/1957 00:00:00,nan,British (White),...,nan,nan,3.0,nan,1.0,nan,1.0,nan,25760.0,10.0
2,RRF02,2023-06-08,2023-08-06 11:45:00,2023-08-06 12:50:00,100.0,SAEC,SAEC,24/11/1975 00:00:00,nan,British (White),...,nan,nan,3.0,nan,1.0,nan,1.0,nan,159753.0,3.0
3,RRF53,2023-06-08,2023-08-06 07:13:00,2023-08-06 17:35:00,103.0,D,D,1974-06-10 00:00:00,nan,British (White),...,nan,nan,3.0,nan,1.0,nan,1.0,nan,76679.0,4.0
4,RRF02,2023-06-08,2023-08-06 08:29:00,2023-08-06 15:34:00,301.0,ENDO,ENDO,21/12/1962 00:00:00,nan,British (White),...,nan,nan,3.0,nan,1.0,nan,1.0,nan,2617.0,9.0


For model (numeric, encoded, scaled):


,specialty_spec_code,duration_elective_wait,patient_age_on_admission,patient_age_on_discharge,spell_episode_los,delayed_discharges_flag,delayed_discharges_no_of_days,readmission_flag_28_days,readmission_flag_30_days,discharge_letter_sent_in_24hrs,...,inj_or_ail_nan,attend_dis_description_Admitted,attend_dis_description_Discharged,attend_dis_description_Transfer,attend_dis_description_Ward,attend_dis_description_nan,location_ECC Awaiting Triage,location_Majors Awaiting Triage,location_Paediatrics Awaiting Triage,location_nan
0,-1.189181,0.0,0.819969,0.819668,-0.642478,0.0,0.0,0.0,0.0,0.0,...,0.561177,-0.576449,-0.14699,-0.009777,-0.057521,0.614271,-0.372088,-0.387414,-0.149431,0.614271
1,-1.138428,0.0,0.390498,0.390247,-0.642478,0.0,0.0,0.0,0.0,0.0,...,0.561177,-0.576449,-0.14699,-0.009777,-0.057521,0.614271,-0.372088,-0.387414,-0.149431,0.614271
2,-1.223017,0.0,-0.382550,-0.382712,-0.642478,0.0,0.0,0.0,0.0,0.0,...,0.561177,-0.576449,-0.14699,-0.009777,-0.057521,0.614271,-0.372088,-0.387414,-0.149431,0.614271
3,-1.197640,0.0,-0.339603,-0.339770,-0.642478,0.0,0.0,0.0,0.0,0.0,...,0.561177,-0.576449,-0.14699,-0.009777,-0.057521,0.614271,-0.372088,-0.387414,-0.149431,0.614271
4,0.477208,0.0,0.175762,0.175536,-0.642478,0.0,0.0,0.0,0.0,0.0,...,0.561177,-0.576449,-0.14699,-0.009777,-0.057521,0.614271,-0.372088,-0.387414,-0.149431,0.614271
